# Generate flux table for pcigale

Test Fors2 IO for `fors2pcigale` package

- Author Sylvie Dagoret-Campagne
- Afflilation : IJCLab/IN2P3/CNRS
- Organisation : LSST-DESC
- creation date : 2024-02-04
- last update : 2024-02-05

  Use ``pcigale`` python environnement

## File to generate for cigale input

In [1]:
# File containing the input data. The columns are 'id' (name of the
# object), 'redshift' (if 0 the distance is assumed to be 10 pc),
# 'distance' (Mpc, optional, if present it will be used in lieu of the
# distance computed from the redshift), the filter names for the fluxes,
# and the filter names with the '_err' suffix for the uncertainties. The
# fluxes and the uncertainties must be in mJy for broadband data and in
# W/m² for emission lines. Fluxes can be positive or negative. Upper
# limits are indicated with a negative value for the uncertainty. In
# case some fluxes are missing for some entries, they can be replaced
# with NaN. This file is optional to generate the configuration file, in
# particular for the savefluxes module.

### name of filters in CIGALE
    redshift distance 
    galex.FUV galex.FUV_err galex.NUV galex.NUV_err 
    sdss.up sdss.up_err sdss.gp sdss.gp_err  sdss.rp sdss.rp_err sdss.ip sdss.ip_err 
   
    vista.vircam.Z                                                                                             
    vista.vircam.Y                                                                                         
    vista.vircam.J                                                                                        
    vista.vircam.H                                                                                       
    vista.vircam.Ks 
    

### emission lines
    line.NII-654.8 line.NII-654.8_err line.NII-658.4 line.NII-658.4_err 
    line.H-alpha line.H-alpha_err 
    line.OIII-500.7 line.OIII-500.7_err 
    line.H-beta line.H-beta_err

## Import

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import sys

/var/folders/79/hrybm_4s0zjd4jsb7lp_trhh0000gp/T/ipykernel_76074/3915411586.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
from fors2pcigale.fors2starlightio import FILENAME_FORS2PHOTOM,FILENAME_STARLIGHT
from fors2pcigale.fors2starlightio import _getPackageDir
from fors2pcigale.fors2starlightio import FULL_FILENAME_FORS2PHOTOM, FULL_FILENAME_STARLIGHT
from fors2pcigale.fors2starlightio import Fors2DataAcess, SLDataAcess
from fors2pcigale.fors2starlightio import convertflambda_to_fnu, flux_norm,ordered_keys

In [4]:
from astropy import units as u
from astropy import constants as const
import numpy as np
from astropy.table import Table
from astropy.io import ascii

## Configuration

In [5]:
_getPackageDir()

'/Users/dagoret/MacOSX/GitHub/LSST/Fors2pcigale/src/fors2pcigale/fors2starlightio'

In [6]:
FILENAME_FORS2PHOTOM

'data/FORS2spectraGalexKidsPhotom.h5'

In [7]:
FILENAME_STARLIGHT

'data/SLspectra.h5'

In [8]:
FULL_FILENAME_FORS2PHOTOM

'/Users/dagoret/MacOSX/GitHub/LSST/Fors2pcigale/src/fors2pcigale/fors2starlightio/data/FORS2spectraGalexKidsPhotom.h5'

In [9]:
!ls /Users/dagoret/MacOSX/GitHub/LSST/Fors2ToStellarPopSynthesis/src/fors2tostellarpopsynthesis/fors2starlightio/data/FORS2spectraGalexKidsPhotom.h5

/Users/dagoret/MacOSX/GitHub/LSST/Fors2ToStellarPopSynthesis/src/fors2tostellarpopsynthesis/fors2starlightio/data/FORS2spectraGalexKidsPhotom.h5


In [10]:
J0 = 3631.0 # convert AB flux in Jansky (1 Jansky = 1e-20 erg/cm2/s)

In [11]:
# select if allow Nans
FLAG_NONAN = False

In [12]:
if FLAG_NONAN: 
    FILENAME_output_ascii = "Fors2Photom_cigaleinput_NONan.txt"
    FILENAME_output_fits = "Fors2Photom_cigaleinput_NONan.fits"
else:
    FILENAME_output_ascii = "Fors2Photom_cigaleinput_WithNan.txt"
    FILENAME_output_fits = "Fors2Photom_cigaleinput_WithNan.fits"

## Initialisation

In [13]:
fors2 = Fors2DataAcess()

In [14]:
list_of_specname = fors2.get_list_of_groupkeys()

In [15]:
for idx,specname in enumerate(list_of_specname):
    print(fors2.get_photfluxes(specname))
    if idx>=1:
        break

(array([6.77413801e-09, 3.48282778e-09,            nan, 7.25752934e-10,
       2.35473479e-09, 4.00341673e-09, 5.63590936e-09, 7.05366452e-09,
       9.07996135e-09, 1.29943452e-08, 1.78360713e-08]), array([1.57494035e-09, 8.18709258e-10,            nan, 4.24668420e-11,
       4.45119259e-11, 1.47622804e-10, 1.83424383e-10, 3.62194927e-10,
       2.93883384e-10, 5.98748019e-10, 6.97714249e-10]))
(array([           nan, 1.35103189e-09, 4.58414065e-10, 6.65999834e-10,
       1.79701182e-09, 3.47277687e-09, 5.01496368e-09, 5.92920575e-09,
       9.30281098e-09, 1.17182542e-08, 1.78949844e-08]), array([           nan, 5.88293786e-10, 1.23928297e-10, 4.13597579e-11,
       4.37090600e-11, 1.45867210e-10, 1.83666168e-10, 3.59159087e-10,
       2.91338699e-10, 5.92897758e-10, 6.88687501e-10]))


In [16]:
for idx,specname in enumerate(list_of_specname):
    print("=========================================================================")
    d = fors2.getattribdata_fromgroup(specname)
    print(d)
    if idx>=1:
        break


OrderedDict([('CLASS_STAR', 0.04272303), ('DECJ2000', -28.447828), ('EXTINCTION_g', 0.05650495), ('EXTINCTION_i', 0.02904796), ('EXTINCTION_r', 0.03908986), ('EXTINCTION_u', 0.07251725), ('FLUXERR_GAAP_H', 594.53876), ('FLUXERR_GAAP_J', 290.61316), ('FLUXERR_GAAP_Ks', 694.3972), ('FLUXERR_GAAP_Y', 356.02774), ('FLUXERR_GAAP_Z', 179.81622), ('FLUXERR_GAAP_g', 3.95e-11), ('FLUXERR_GAAP_i', 1.56e-10), ('FLUXERR_GAAP_r', 4.15e-11), ('FLUXERR_GAAP_u', nan), ('FLUX_GAAP_H', 12903.003), ('FLUX_GAAP_J', 8978.914), ('FLUX_GAAP_Ks', 17751.28), ('FLUX_GAAP_Y', 6933.5605), ('FLUX_GAAP_Z', 5525.0454), ('FLUX_GAAP_g', 6.7505e-10), ('FLUX_GAAP_i', 4.2306e-09), ('FLUX_GAAP_r', 2.1954e-09), ('FLUX_GAAP_u', nan), ('FLUX_RADIUS', 2.967994), ('ID', 'KiDSDR4 J005407.202-282652.18 '), ('KIDS_TILE', 'KIDS_13.5_-28.2 '), ('MAGERR_GAAP_H', 0.0500281), ('MAGERR_GAAP_J', 0.0351411), ('MAGERR_GAAP_Ks', 0.042472), ('MAGERR_GAAP_Y', 0.0557509), ('MAGERR_GAAP_Z', 0.035336), ('MAGERR_GAAP_g', 0.06353097), ('MAGERR_GA

## Build dataframe for output file

In [17]:
df_f = pd.DataFrame(columns=['redshift','fuv_flux','fuv_fluxerr',
                             'nuv_flux','nuv_fluxerr',
                             'FLUX_GAAP_u','FLUXERR_GAAP_u',
                             'FLUX_GAAP_g','FLUXERR_GAAP_g',
                             'FLUX_GAAP_r','FLUXERR_GAAP_r',
                             'FLUX_GAAP_i','FLUXERR_GAAP_i',
                             'FLUX_GAAP_Z','FLUXERR_GAAP_Z',
                             'FLUX_GAAP_Y','FLUXERR_GAAP_Y',
                             'FLUX_GAAP_J','FLUXERR_GAAP_J',
                             'FLUX_GAAP_H','FLUXERR_GAAP_H',
                             'FLUX_GAAP_Ks','FLUXERR_GAAP_Ks',
                            ], index=list_of_specname)
df_m = pd.DataFrame(columns=['redshift','fuv_mag','fuv_magerr',
                             'nuv_mag','nuv_magerr',
                             'MAG_GAAP_u','MAGERR_GAAP_u',
                             'MAG_GAAP_g','MAGERR_GAAP_g',
                             'MAG_GAAP_r','MAGERR_GAAP_r',
                             'MAG_GAAP_i','MAGERR_GAAP_i',
                             'MAG_GAAP_Z','MAGERR_GAAP_Z',
                             'MAG_GAAP_Y','MAGERR_GAAP_Y',
                             'MAG_GAAP_J','MAGERR_GAAP_J',
                             'MAG_GAAP_H','MAGERR_GAAP_H',
                             'MAG_GAAP_Ks','MAGERR_GAAP_Ks',
                            ], index=list_of_specname)

### Set the index name
- the dataframe index have the spec name. The index bears the name id as required by pcigale

In [18]:
df_f.index.name='id'
df_m.index.name='id'

In [19]:
# apparently flux are in micro-Jansky
for idx,specname in enumerate(list_of_specname):
    d = fors2.getattribdata_fromgroup(specname)
    df_f.loc[specname] = pd.Series({'redshift':d['redshift'], 
                                     'fuv_flux':d['fuv_flux'], 
                                     'fuv_fluxerr':d['fuv_fluxerr'],
                                     'nuv_flux':d['nuv_flux'], 
                                     'nuv_fluxerr':d['nuv_fluxerr'],
                                     'FLUX_GAAP_u':d['FLUX_GAAP_u'],
                                     'FLUXERR_GAAP_u':d['FLUXERR_GAAP_u'],
                                     'FLUX_GAAP_g':d['FLUX_GAAP_g'],
                                     'FLUXERR_GAAP_g':d['FLUXERR_GAAP_g'],
                                     'FLUX_GAAP_r':d['FLUX_GAAP_r'],
                                     'FLUXERR_GAAP_r':d['FLUXERR_GAAP_r'],
                                     'FLUX_GAAP_i':d['FLUX_GAAP_i'],
                                     'FLUXERR_GAAP_i':d['FLUXERR_GAAP_i'],
                                     'FLUX_GAAP_Z':d['FLUX_GAAP_Z'],
                                     'FLUXERR_GAAP_Z':d['FLUXERR_GAAP_Z'],
                                     'FLUX_GAAP_Y':d['FLUX_GAAP_Y'],
                                     'FLUXERR_GAAP_Y':d['FLUXERR_GAAP_Y'],
                                     'FLUX_GAAP_J':d['FLUX_GAAP_J'],
                                     'FLUXERR_GAAP_J':d['FLUXERR_GAAP_J'],
                                     'FLUX_GAAP_H':d['FLUX_GAAP_H'],
                                     'FLUXERR_GAAP_H':d['FLUXERR_GAAP_H'],
                                     'FLUX_GAAP_Ks':d['FLUX_GAAP_Ks'], 
                                     'FLUXERR_GAAP_Ks':d['FLUXERR_GAAP_Ks'], 
                                   })

In [20]:
# magnitudes are AB
for idx,specname in enumerate(list_of_specname):
    d = fors2.getattribdata_fromgroup(specname)
    df_m.loc[specname] = pd.Series({'redshift':d['redshift'], 
                                     'fuv_mag':d['fuv_mag'], 
                                     'fuv_magerr':d['fuv_magerr'],
                                     'nuv_mag':d['nuv_mag'], 
                                     'nuv_magerr':d['nuv_magerr'],
                                     'MAG_GAAP_u':d['MAG_GAAP_u'],
                                     'MAGERR_GAAP_u':d['MAGERR_GAAP_u'],
                                     'MAG_GAAP_g':d['MAG_GAAP_g'],
                                     'MAGERR_GAAP_g':d['MAGERR_GAAP_g'],
                                     'MAG_GAAP_r':d['MAG_GAAP_r'],
                                     'MAGERR_GAAP_r':d['MAGERR_GAAP_r'],
                                     'MAG_GAAP_i':d['MAG_GAAP_i'],
                                     'MAGERR_GAAP_i':d['MAGERR_GAAP_i'],
                                     'MAG_GAAP_Z':d['MAG_GAAP_Z'],
                                     'MAGERR_GAAP_Z':d['MAGERR_GAAP_Z'],
                                     'MAG_GAAP_Y':d['MAG_GAAP_Y'],
                                     'MAGERR_GAAP_Y':d['MAGERR_GAAP_Y'],
                                     'MAG_GAAP_J':d['MAG_GAAP_J'],
                                     'MAGERR_GAAP_J':d['MAGERR_GAAP_J'],
                                     'MAG_GAAP_H':d['MAG_GAAP_H'],
                                     'MAGERR_GAAP_H':d['MAGERR_GAAP_H'],
                                     'MAG_GAAP_Ks':d['MAG_GAAP_Ks'],
                                     'MAGERR_GAAP_Ks':d['MAGERR_GAAP_Ks'],
                                   })

In [21]:
# flux in micro-Jansky
df_f.head()

,redshift,fuv_flux,fuv_fluxerr,nuv_flux,nuv_fluxerr,FLUX_GAAP_u,FLUXERR_GAAP_u,FLUX_GAAP_g,FLUXERR_GAAP_g,FLUX_GAAP_r,...,FLUX_GAAP_Z,FLUXERR_GAAP_Z,FLUX_GAAP_Y,FLUXERR_GAAP_Y,FLUX_GAAP_J,FLUXERR_GAAP_J,FLUX_GAAP_H,FLUXERR_GAAP_H,FLUX_GAAP_Ks,FLUXERR_GAAP_Ks
id,,,,,,,,,,,,,,,,,,,,,
SPEC2,0.34555,24.59552,5.7169,12.645368,2.971828,NaN,NaN,0.0,0.0,0.0,...,5525.0454,179.81622,6933.5605,356.02774,8978.914,290.61316,12903.003,594.53876,17751.28,694.3972
SPEC3,0.6866,NaN,NaN,4.905293,2.135443,0.0,0.0,0.0,0.0,0.0,...,4912.493164,179.913452,5824.850098,352.837769,9195.825195,287.987885,11632.511719,588.559448,17807.927734,685.337463
SPEC9,0.29266,NaN,NaN,9.029878,2.996898,NaN,NaN,0.0,0.0,0.0,...,3194.795654,185.288483,3145.443359,330.740845,4558.929688,268.486725,5803.071777,579.864075,6177.433594,606.197693
SPEC13,0.6887,NaN,NaN,3.221977,1.612464,0.0,0.0,0.0,0.0,0.0,...,1140.422485,180.381531,442.328766,323.638702,1297.0,260.288177,NaN,NaN,NaN,NaN
SPEC19,0.48395,NaN,NaN,12.55755,3.31995,0.0,0.0,0.0,0.0,0.0,...,3555.818848,123.689148,4437.370117,238.951889,5302.674805,225.177338,6213.995117,544.024658,8170.591309,507.931976


In [22]:
if FLAG_NONAN:
    df_f.dropna(inplace=True)

In [23]:
# magnitude AB
df_m.head()

,redshift,fuv_mag,fuv_magerr,nuv_mag,nuv_magerr,MAG_GAAP_u,MAGERR_GAAP_u,MAG_GAAP_g,MAGERR_GAAP_g,MAG_GAAP_r,...,MAG_GAAP_Z,MAGERR_GAAP_Z,MAG_GAAP_Y,MAGERR_GAAP_Y,MAG_GAAP_J,MAGERR_GAAP_J,MAG_GAAP_H,MAGERR_GAAP_H,MAG_GAAP_Ks,MAGERR_GAAP_Ks
id,,,,,,,,,,,,,,,,,,,,,
SPEC2,0.34555,20.422865,0.252426,21.14517,0.255224,NaN,NaN,22.848028,0.063531,21.570145,...,20.62259,0.035336,20.378963,0.055751,20.10479,0.035141,19.715614,0.050028,19.371752,0.042472
SPEC3,0.6866,NaN,NaN,22.173336,0.472773,23.346855,0.293519,22.941315,0.067426,21.863623,...,20.749331,0.039764,20.567509,0.065768,20.078465,0.034002,19.827843,0.054934,19.368172,0.041784
SPEC9,0.29266,NaN,NaN,21.510794,0.360429,NaN,NaN,23.450844,0.091752,21.977757,...,21.216759,0.062969,21.236706,0.114164,20.840393,0.063942,20.583025,0.108491,20.517725,0.106544
SPEC13,0.6887,NaN,NaN,22.629692,0.543497,23.861921,0.408772,23.616484,0.094022,23.223907,...,22.334183,0.171732,23.365664,0.794401,22.204744,0.217891,NaN,NaN,NaN,NaN
SPEC19,0.48395,NaN,NaN,21.152737,0.287115,24.186991,0.698657,23.808002,0.131907,22.011513,...,21.100159,0.037767,20.862761,0.058467,20.676168,0.046106,20.508581,0.095054,20.214029,0.067496


### apparently redshift appears to be an object not a float.

then convert it

In [24]:
df_m['redshift'] = df_m['redshift'].astype(float)
df_f['redshift'] = df_f['redshift'].astype(float)

In [25]:
if FLAG_NONAN:
    df_m.dropna(inplace=True)

In [26]:
df_f.head()

,redshift,fuv_flux,fuv_fluxerr,nuv_flux,nuv_fluxerr,FLUX_GAAP_u,FLUXERR_GAAP_u,FLUX_GAAP_g,FLUXERR_GAAP_g,FLUX_GAAP_r,...,FLUX_GAAP_Z,FLUXERR_GAAP_Z,FLUX_GAAP_Y,FLUXERR_GAAP_Y,FLUX_GAAP_J,FLUXERR_GAAP_J,FLUX_GAAP_H,FLUXERR_GAAP_H,FLUX_GAAP_Ks,FLUXERR_GAAP_Ks
id,,,,,,,,,,,,,,,,,,,,,
SPEC2,0.34555,24.59552,5.7169,12.645368,2.971828,NaN,NaN,0.0,0.0,0.0,...,5525.0454,179.81622,6933.5605,356.02774,8978.914,290.61316,12903.003,594.53876,17751.28,694.3972
SPEC3,0.68660,NaN,NaN,4.905293,2.135443,0.0,0.0,0.0,0.0,0.0,...,4912.493164,179.913452,5824.850098,352.837769,9195.825195,287.987885,11632.511719,588.559448,17807.927734,685.337463
SPEC9,0.29266,NaN,NaN,9.029878,2.996898,NaN,NaN,0.0,0.0,0.0,...,3194.795654,185.288483,3145.443359,330.740845,4558.929688,268.486725,5803.071777,579.864075,6177.433594,606.197693
SPEC13,0.68870,NaN,NaN,3.221977,1.612464,0.0,0.0,0.0,0.0,0.0,...,1140.422485,180.381531,442.328766,323.638702,1297.0,260.288177,NaN,NaN,NaN,NaN
SPEC19,0.48395,NaN,NaN,12.55755,3.31995,0.0,0.0,0.0,0.0,0.0,...,3555.818848,123.689148,4437.370117,238.951889,5302.674805,225.177338,6213.995117,544.024658,8170.591309,507.931976


In [27]:
df_m.head()

,redshift,fuv_mag,fuv_magerr,nuv_mag,nuv_magerr,MAG_GAAP_u,MAGERR_GAAP_u,MAG_GAAP_g,MAGERR_GAAP_g,MAG_GAAP_r,...,MAG_GAAP_Z,MAGERR_GAAP_Z,MAG_GAAP_Y,MAGERR_GAAP_Y,MAG_GAAP_J,MAGERR_GAAP_J,MAG_GAAP_H,MAGERR_GAAP_H,MAG_GAAP_Ks,MAGERR_GAAP_Ks
id,,,,,,,,,,,,,,,,,,,,,
SPEC2,0.34555,20.422865,0.252426,21.14517,0.255224,NaN,NaN,22.848028,0.063531,21.570145,...,20.62259,0.035336,20.378963,0.055751,20.10479,0.035141,19.715614,0.050028,19.371752,0.042472
SPEC3,0.68660,NaN,NaN,22.173336,0.472773,23.346855,0.293519,22.941315,0.067426,21.863623,...,20.749331,0.039764,20.567509,0.065768,20.078465,0.034002,19.827843,0.054934,19.368172,0.041784
SPEC9,0.29266,NaN,NaN,21.510794,0.360429,NaN,NaN,23.450844,0.091752,21.977757,...,21.216759,0.062969,21.236706,0.114164,20.840393,0.063942,20.583025,0.108491,20.517725,0.106544
SPEC13,0.68870,NaN,NaN,22.629692,0.543497,23.861921,0.408772,23.616484,0.094022,23.223907,...,22.334183,0.171732,23.365664,0.794401,22.204744,0.217891,NaN,NaN,NaN,NaN
SPEC19,0.48395,NaN,NaN,21.152737,0.287115,24.186991,0.698657,23.808002,0.131907,22.011513,...,21.100159,0.037767,20.862761,0.058467,20.676168,0.046106,20.508581,0.095054,20.214029,0.067496


## Conversion functions

In [28]:
# Convert AB to milli Jansky
conv_magABtoFluxmlJy = lambda x: np.power(10.,-0.4*x)*J0*1000.
conv_magerrABtoErrFluxmlJy = lambda x: np.log(10)/2.5*x[1]*conv_magABtoFluxmlJy(x[0])
# Convert AB to micro Jansky
conv_magABtoFluxmcJy = lambda x: np.power(10.,-0.4*x)*J0*1000000.
conv_magerrABtoErrFluxmcJy = lambda x: np.log(10)/2.5*x[1]*conv_magABtoFluxmcJy(x[0]) 

### Check the conversion function by comparing flux and magnitude dataframe with calculation
- check done in micro Jansky

In [29]:
df_m["fuv_mag"].apply(conv_magABtoFluxmcJy).head() 

id
SPEC2     24.596895
SPEC3           NaN
SPEC9           NaN
SPEC13          NaN
SPEC19          NaN
Name: fuv_mag, dtype: float64

In [30]:
df_m[["fuv_mag","fuv_magerr"]].apply(conv_magerrABtoErrFluxmcJy,axis=1).head()

/var/folders/79/hrybm_4s0zjd4jsb7lp_trhh0000gp/T/ipykernel_76074/3128002138.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conv_magerrABtoErrFluxmcJy = lambda x: np.log(10)/2.5*x[1]*conv_magABtoFluxmcJy(x[0])


id
SPEC2     5.718608
SPEC3          NaN
SPEC9          NaN
SPEC13         NaN
SPEC19         NaN
dtype: float64

### Does the final conversion and drop unnecessary columns

- calculate flux and flux errors in milli-Jansky as required by pcigale

In [31]:
df_m["galex.FUV"] = df_m["fuv_mag"].apply(conv_magABtoFluxmlJy)
df_m["galex.FUV_err"] = df_m[["fuv_mag","fuv_magerr"]].apply(conv_magerrABtoErrFluxmlJy,axis=1)
df_m["galex.NUV"] = df_m["nuv_mag"].apply(conv_magABtoFluxmlJy)
df_m["galex.NUV_err"] = df_m[["nuv_mag","nuv_magerr"]].apply(conv_magerrABtoErrFluxmlJy,axis=1)

/var/folders/79/hrybm_4s0zjd4jsb7lp_trhh0000gp/T/ipykernel_76074/3128002138.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conv_magerrABtoErrFluxmlJy = lambda x: np.log(10)/2.5*x[1]*conv_magABtoFluxmlJy(x[0])
/var/folders/79/hrybm_4s0zjd4jsb7lp_trhh0000gp/T/ipykernel_76074/3128002138.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conv_magerrABtoErrFluxmlJy = lambda x: np.log(10)/2.5*x[1]*conv_magABtoFluxmlJy(x[0])


In [32]:
df_m.drop(["fuv_mag","fuv_magerr","nuv_mag","nuv_magerr"], axis=1, inplace=True)

In [33]:
df_m["sdss.up"] =  df_m["MAG_GAAP_u"].apply(conv_magABtoFluxmlJy)
df_m["sdss.up_err"] = df_m[["MAG_GAAP_u","MAGERR_GAAP_u"]].apply(conv_magerrABtoErrFluxmlJy,axis=1)
df_m["sdss.gp"] =  df_m["MAG_GAAP_g"].apply(conv_magABtoFluxmlJy)
df_m["sdss.gp_err"] = df_m[["MAG_GAAP_g","MAGERR_GAAP_g"]].apply(conv_magerrABtoErrFluxmlJy,axis=1)
df_m["sdss.rp"] =  df_m["MAG_GAAP_r"].apply(conv_magABtoFluxmlJy)
df_m["sdss.rp_err"] = df_m[["MAG_GAAP_r","MAGERR_GAAP_r"]].apply(conv_magerrABtoErrFluxmlJy,axis=1)
df_m["sdss.ip"] =  df_m["MAG_GAAP_i"].apply(conv_magABtoFluxmlJy)
df_m["sdss.ip_err"] = df_m[["MAG_GAAP_i","MAGERR_GAAP_i"]].apply(conv_magerrABtoErrFluxmlJy,axis=1)

/var/folders/79/hrybm_4s0zjd4jsb7lp_trhh0000gp/T/ipykernel_76074/3128002138.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conv_magerrABtoErrFluxmlJy = lambda x: np.log(10)/2.5*x[1]*conv_magABtoFluxmlJy(x[0])
/var/folders/79/hrybm_4s0zjd4jsb7lp_trhh0000gp/T/ipykernel_76074/3128002138.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conv_magerrABtoErrFluxmlJy = lambda x: np.log(10)/2.5*x[1]*conv_magABtoFluxmlJy(x[0])
/var/folders/79/hrybm_4s0zjd4jsb7lp_trhh0000gp/T/ipykernel_76074/3128002138.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will

In [34]:
df_m.drop(["MAG_GAAP_u","MAGERR_GAAP_u", "MAG_GAAP_g","MAGERR_GAAP_g",
           "MAG_GAAP_r","MAGERR_GAAP_r", "MAG_GAAP_i","MAGERR_GAAP_i",
          ], axis=1, inplace=True)

In [35]:
df_m["vista.vircam.Z"] =  df_m["MAG_GAAP_Z"].apply(conv_magABtoFluxmlJy)
df_m["vista.vircam.Z_err"] = df_m[["MAG_GAAP_Z","MAGERR_GAAP_Z"]].apply(conv_magerrABtoErrFluxmlJy,axis=1)
df_m["vista.vircam.Y"] =  df_m["MAG_GAAP_Y"].apply(conv_magABtoFluxmlJy)
df_m["vista.vircam.Y_err"] = df_m[["MAG_GAAP_Y","MAGERR_GAAP_Y"]].apply(conv_magerrABtoErrFluxmlJy,axis=1)
df_m["vista.vircam.J"] =  df_m["MAG_GAAP_J"].apply(conv_magABtoFluxmlJy)
df_m["vista.vircam.J_err"] = df_m[["MAG_GAAP_J","MAGERR_GAAP_J"]].apply(conv_magerrABtoErrFluxmlJy,axis=1)
df_m["vista.vircam.H"] =  df_m["MAG_GAAP_H"].apply(conv_magABtoFluxmlJy)
df_m["vista.vircam.H_err"] = df_m[["MAG_GAAP_H","MAGERR_GAAP_H"]].apply(conv_magerrABtoErrFluxmlJy,axis=1)
df_m["vista.vircam.Ks"] =  df_m["MAG_GAAP_Ks"].apply(conv_magABtoFluxmlJy)
df_m["vista.vircam.Ks_err"] = df_m[["MAG_GAAP_Ks","MAGERR_GAAP_Ks"]].apply(conv_magerrABtoErrFluxmlJy,axis=1)

/var/folders/79/hrybm_4s0zjd4jsb7lp_trhh0000gp/T/ipykernel_76074/3128002138.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conv_magerrABtoErrFluxmlJy = lambda x: np.log(10)/2.5*x[1]*conv_magABtoFluxmlJy(x[0])
/var/folders/79/hrybm_4s0zjd4jsb7lp_trhh0000gp/T/ipykernel_76074/3128002138.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conv_magerrABtoErrFluxmlJy = lambda x: np.log(10)/2.5*x[1]*conv_magABtoFluxmlJy(x[0])
/var/folders/79/hrybm_4s0zjd4jsb7lp_trhh0000gp/T/ipykernel_76074/3128002138.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will

In [36]:
df_m.drop(["MAG_GAAP_Z","MAGERR_GAAP_Z","MAG_GAAP_Y","MAGERR_GAAP_Y",
           "MAG_GAAP_J","MAGERR_GAAP_J","MAG_GAAP_H","MAGERR_GAAP_H",
           "MAG_GAAP_Ks","MAGERR_GAAP_Ks"], axis=1, inplace=True)

### Now have the final format

In [37]:
df_m.reset_index(inplace=True)
df_m

,id,redshift,galex.FUV,galex.FUV_err,galex.NUV,galex.NUV_err,sdss.up,sdss.up_err,sdss.gp,sdss.gp_err,...,vista.vircam.Z,vista.vircam.Z_err,vista.vircam.Y,vista.vircam.Y_err,vista.vircam.J,vista.vircam.J_err,vista.vircam.H,vista.vircam.H_err,vista.vircam.Ks,vista.vircam.Ks_err
0,SPEC2,0.34555,0.024597,0.005719,0.012646,0.002973,NaN,NaN,0.002635,0.000154,...,0.020464,0.000666,0.025612,0.001315,0.032969,0.001067,0.047182,0.002174,0.064763,0.002533
1,SPEC3,0.68660,NaN,NaN,0.004906,0.002136,0.001665,0.000450,0.002418,0.000150,...,0.018209,0.000667,0.021529,0.001304,0.033779,0.001058,0.042549,0.002153,0.064977,0.002501
2,SPEC9,0.29266,NaN,NaN,0.009030,0.002998,NaN,NaN,0.001512,0.000128,...,0.011839,0.000687,0.011624,0.001222,0.016744,0.000986,0.021223,0.002121,0.022539,0.002212
3,SPEC13,0.68870,NaN,NaN,0.003222,0.001613,0.001036,0.000390,0.001298,0.000112,...,0.004230,0.000669,0.001636,0.001197,0.004766,0.000956,NaN,NaN,NaN,NaN
4,SPEC19,0.48395,NaN,NaN,0.012558,0.003321,0.000768,0.000494,0.001088,0.000132,...,0.013181,0.000459,0.016403,0.000883,0.019479,0.000827,0.022730,0.001990,0.029814,0.001853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,SPEC734,0.23012,0.055610,0.007888,0.921759,0.019398,0.005131,0.000423,0.012202,0.000125,...,0.047212,0.000365,0.053527,0.000900,0.070059,0.000723,0.088349,0.001687,0.085755,0.001807
546,SPEC735,0.36169,0.022658,0.005798,0.030879,0.008456,0.001204,0.000374,0.011294,0.000127,...,0.103040,0.000380,0.126189,0.000939,0.157608,0.000760,0.218320,0.001754,0.232103,0.001904
547,SPEC736,0.55777,0.055610,0.007888,0.921759,0.019398,0.001954,0.000446,0.002162,0.000138,...,0.007443,0.000402,0.009944,0.000822,0.010314,0.000682,0.007103,0.001572,0.011187,0.001665
548,SPEC737,0.52001,0.011013,0.003888,0.007556,0.002762,0.001446,0.000444,0.002470,0.000121,...,0.017407,0.000391,0.025460,0.000799,0.030801,0.000659,0.042344,0.001534,0.049136,0.001586


## Output ascii file

- convert as an astropy table to have the output formatting

In [38]:
#df_m.to_csv('foo.txt', sep='\t')

In [39]:
t = Table.from_pandas(df_m)

In [40]:
FLAG_FORMAT = True
if FLAG_FORMAT:
    t.info.format = '.5f'
    colnames = t.colnames
    ncols = len(colnames)
    t['redshift'].info.format = '.4f'

    for idx in range(2,ncols):
        colname = colnames[idx]
        t[colname].info.format = '.6f'

In [41]:
t

id,redshift,galex.FUV,galex.FUV_err,galex.NUV,galex.NUV_err,sdss.up,sdss.up_err,sdss.gp,sdss.gp_err,sdss.rp,sdss.rp_err,sdss.ip,sdss.ip_err,vista.vircam.Z,vista.vircam.Z_err,vista.vircam.Y,vista.vircam.Y_err,vista.vircam.J,vista.vircam.J_err,vista.vircam.H,vista.vircam.H_err,vista.vircam.Ks,vista.vircam.Ks_err
str7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
SPEC2,0.3456,0.024597,0.005719,0.012646,0.002973,--,--,0.002635,0.000154,0.008550,0.000162,0.014536,0.000536,0.020464,0.000666,0.025612,0.001315,0.032969,0.001067,0.047182,0.002174,0.064763,0.002533
SPEC3,0.6866,--,--,0.004906,0.002136,0.001665,0.000450,0.002418,0.000150,0.006525,0.000159,0.012610,0.000530,0.018209,0.000667,0.021529,0.001304,0.033779,0.001058,0.042549,0.002153,0.064977,0.002501
SPEC9,0.2927,--,--,0.009030,0.002998,--,--,0.001512,0.000128,0.005874,0.000135,0.008643,0.000474,0.011839,0.000687,0.011624,0.001222,0.016744,0.000986,0.021223,0.002121,0.022539,0.002212
SPEC13,0.6887,--,--,0.003222,0.001613,0.001036,0.000390,0.001298,0.000112,0.001864,0.000116,0.002078,0.000520,0.004230,0.000669,0.001636,0.001197,0.004766,0.000956,--,--,--,--
SPEC19,0.4839,--,--,0.012558,0.003321,0.000768,0.000494,0.001088,0.000132,0.005694,0.000138,0.009228,0.000526,0.013181,0.000459,0.016403,0.000883,0.019479,0.000827,0.022730,0.001990,0.029814,0.001853
SPEC24,0.4661,--,--,0.004014,0.002039,0.000718,0.000464,0.001009,0.000128,0.004204,0.000136,0.008644,0.000517,0.012985,0.000408,0.016747,0.001319,0.025418,0.001078,0.032555,0.002240,0.043469,0.002496
SPEC25,0.4481,--,--,0.012558,0.003321,0.001499,0.000426,0.001288,0.000116,0.003353,0.000122,0.004357,0.000516,0.005358,0.000370,0.006911,0.000712,0.009125,0.001036,0.006911,0.002135,0.006642,0.001464
SPEC30,0.2903,--,--,0.007294,0.003166,0.001786,0.000486,0.003928,0.000135,0.009402,0.000144,0.013889,0.000540,0.018333,0.000673,0.019506,0.001313,0.025301,0.001088,0.032621,0.002245,0.038438,0.002562
SPEC31,0.6327,--,--,0.004949,0.001965,0.001030,0.000448,0.002151,0.000122,0.006687,0.000129,0.014938,0.000496,0.022387,0.000699,0.034818,0.001275,0.045401,0.001034,0.068712,0.002175,0.088688,0.002412


### Format
https://docs.astropy.org/en/stable/io/unified.html
https://docs.astropy.org/en/stable/io/ascii/read.html#replace-bad-or-missing-values

## Save file


crash in saving tthe table

In [42]:
if FLAG_NONAN: 
    t.write(FILENAME_output_ascii ,format="ascii.tab",overwrite=True)
    t.write(FILENAME_output_fits,overwrite=True)
    #t.write(FILENAME_output_ascii ,format="ascii.fixed_width",overwrite=True)
    
else:
    #t['galex.FUV'].fill_value = np.NaN
    t.write(FILENAME_output_ascii ,format="ascii.tab",overwrite=True)
    t.write(FILENAME_output_fits ,overwrite=True)
    #t.write(FILENAME_output_ascii ,format="ascii.fixed_width",overwrite=True)

## Read again the file

In [43]:
t_in = Table.t = Table.read(FILENAME_output_fits)

In [44]:
t_in

id,redshift,galex.FUV,galex.FUV_err,galex.NUV,galex.NUV_err,sdss.up,sdss.up_err,sdss.gp,sdss.gp_err,sdss.rp,sdss.rp_err,sdss.ip,sdss.ip_err,vista.vircam.Z,vista.vircam.Z_err,vista.vircam.Y,vista.vircam.Y_err,vista.vircam.J,vista.vircam.J_err,vista.vircam.H,vista.vircam.H_err,vista.vircam.Ks,vista.vircam.Ks_err
bytes7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
SPEC2,0.3456,0.024597,0.005719,0.012646,0.002973,--,--,0.002635,0.000154,0.008550,0.000162,0.014536,0.000536,0.020464,0.000666,0.025612,0.001315,0.032969,0.001067,0.047182,0.002174,0.064763,0.002533
SPEC3,0.6866,--,--,0.004906,0.002136,0.001665,0.000450,0.002418,0.000150,0.006525,0.000159,0.012610,0.000530,0.018209,0.000667,0.021529,0.001304,0.033779,0.001058,0.042549,0.002153,0.064977,0.002501
SPEC9,0.2927,--,--,0.009030,0.002998,--,--,0.001512,0.000128,0.005874,0.000135,0.008643,0.000474,0.011839,0.000687,0.011624,0.001222,0.016744,0.000986,0.021223,0.002121,0.022539,0.002212
SPEC13,0.6887,--,--,0.003222,0.001613,0.001036,0.000390,0.001298,0.000112,0.001864,0.000116,0.002078,0.000520,0.004230,0.000669,0.001636,0.001197,0.004766,0.000956,--,--,--,--
SPEC19,0.4839,--,--,0.012558,0.003321,0.000768,0.000494,0.001088,0.000132,0.005694,0.000138,0.009228,0.000526,0.013181,0.000459,0.016403,0.000883,0.019479,0.000827,0.022730,0.001990,0.029814,0.001853
SPEC24,0.4661,--,--,0.004014,0.002039,0.000718,0.000464,0.001009,0.000128,0.004204,0.000136,0.008644,0.000517,0.012985,0.000408,0.016747,0.001319,0.025418,0.001078,0.032555,0.002240,0.043469,0.002496
SPEC25,0.4481,--,--,0.012558,0.003321,0.001499,0.000426,0.001288,0.000116,0.003353,0.000122,0.004357,0.000516,0.005358,0.000370,0.006911,0.000712,0.009125,0.001036,0.006911,0.002135,0.006642,0.001464
SPEC30,0.2903,--,--,0.007294,0.003166,0.001786,0.000486,0.003928,0.000135,0.009402,0.000144,0.013889,0.000540,0.018333,0.000673,0.019506,0.001313,0.025301,0.001088,0.032621,0.002245,0.038438,0.002562
SPEC31,0.6327,--,--,0.004949,0.001965,0.001030,0.000448,0.002151,0.000122,0.006687,0.000129,0.014938,0.000496,0.022387,0.000699,0.034818,0.001275,0.045401,0.001034,0.068712,0.002175,0.088688,0.002412
